In [61]:
import pandas as pd
pd.set_option('display.max_columns', None)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
appended_data = []
for year in range(1985, 2020):
    result = pd.DataFrame()
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(year)
    url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(year)
    url3 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    #Extract data from first url
    temp = pd.read_html(url, header=[1])
    temp[0].dropna(axis=1,inplace=True)
    temp = temp[0][0:10]
    temp = temp.drop('Rank', 1)
    temp.insert(1, 'Year', year)
    temp.insert(2, 'MVP', [1,0,0,0,0,0,0,0,0,0])
    #Extract data from second url
    temp2 = pd.read_html(url2)
    temp2 = temp2[0].dropna(axis=1)
    temp2['Player'] = temp2['Player'].str.replace('*','')
    result = temp.merge(temp2, how='inner', on=['Player'])
    #Extract data from third url
    temp3 = pd.read_html(url3)
    temp3 = temp3[0].dropna(axis=1)
    temp3['Player'] = temp3['Player'].str.replace('*','')
    result = result.merge(temp3, how='inner', on=['Player'])
    #add to list of data
    appended_data.append(result)
df = pd.concat(appended_data)
df.to_csv('csv/historical_mvp_data.csv')

In [2]:
# from urllib.request import urlopen
# from bs4 import BeautifulSoup
# import pandas as pd

# # NBA season we will be analyzing
# year = 1985
# # URL page we will scraping (see image above)
# url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(year)
# html = urlopen(url)
# soup = BeautifulSoup(html)

# # use findALL() to get the column headers
# soup.findAll('tr', limit=2)# use getText()to extract the text we need into a list
# headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
# headers = headers[1:]
# headers

# # # avoid the first header row
# # rows = soup.findAll('tr')[1:]
# # player_stats = [[td.getText() for td in rows[i].findAll('td')]
# #             for i in range(len(rows))]

# # stats = pd.DataFrame(player_stats, columns = headers)
# # stats.head(10)